In [1]:
# import libraries 
import pandas as pd
import numpy as np
import subprocess
import re

In [2]:
#read the clinvar with base before and after file as tsv 
# old ver: "/private5/Projects/dadush/clinvar_base_editing/filter_to_ff/FF_14_2_23_with_nAn_with_base_before_and_after.tsv"
# "/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_with_nAn_values_15_3_23_with_base_before_and_after.tsv"
clinvar_path = "/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_with_nAn_values_15_3_23_ver_2_with_base_before_and_after.tsv" 

#col_names = ['chrom','chromStart','chromEnd','name','score','strand','thickStart','thickEnd','reserved','blockCount','blockSizes','chromStarts','origName','clinSign','reviewStatus','type','geneId','molConseq','snpId','nsvId','rcvAcc','testedInGtr','phenotypeList','phenotype','origin','assembly','cytogenetic','_jsonHgvsTable','_hgvsProt','numSubmit','lastEval','guidelines','otherIds','_mouseOver','_clinSignCode','_originCode','_allTypeCode','_varLen','_starCount','_variantId','_dbVarSsvId']
clinvar_df = pd.read_csv(clinvar_path, sep='\t')#,names=col_names)
clinvar_df = clinvar_df.rename({'NA':'base_before','NA.1':'base_after','NA.2':'Strand'}, axis=1)
pd.set_option('display.max_columns', None)
clinvar_df

/tmp/ipykernel_16912/1994752395.py:7: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  clinvar_df = pd.read_csv(clinvar_path, sep='\t')#,names=col_names)


,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand
0,chr1,943994,943995,C>T,1,.,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,+
1,chr1,964511,964512,C>A,0,.,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,+
2,chr1,1013982,1013983,G>A,1,.,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,+
3,chr1,1014142,1014143,C>T,0,.,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,+
4,chr1,1014227,1014228,G>A,1,.,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser83Asn""], [""c.248G>A"",...",nAn,4,"Sep 22,2021",nAn,ClinGen:CA507658,"NM_005101.4(ISG15):c.248G>A (p.Ser83Asn), Type...",CF,GERM,SUBST,1,1,402986,nAn,1,1014228.0,402986.0,G,A,.,.,AF_ESP=0.40158;AF_EXAC=0.37025;AF_TGP=0.33886;...,chr1,Conflicting_interpretations_of_pathogenicity,"Benign(3),Pathogenic(1)",yes,A,C,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

check if all rows has base before and after and strand

In [5]:
# Checking if there is an actual junk value, this is done for all three columns
part_df = clinvar_df[clinvar_df['Strand']=='NA']
part_df

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand


In [21]:
# Checking whether there is a junk value is done for all three columns
part_df = clinvar_df[clinvar_df['Strand'].isna()]
part_df

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand


In [22]:
#Checking that the distribution of the results does indeed contain only possible results that come out of the script and that results from other columns were not mistakenly entered
clinvar_df.groupby(['Strand'])['Strand'].count()

Strand
+    47962
-    50865
Name: Strand, dtype: int64

In [23]:
# Visual inspection done in the UCSC db
clinvar_df[clinvar_df['base_after']=='A']

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand
3,chr1,1014142,1014143,C>T,0,.,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,+
5,chr1,1014358,1014359,G>T,0,.,1014358,1014359,"210,0,0",1,1,0,161454|NM_005101.4(ISG15):c.379G>T (p.Glu127Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,672601312.0,nAn,RCV000148988,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.379G>T"", ""p.Glu127Ter""], [""g.18222G>T"", ""...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA174067, <a target=_blank href='http:...","NM_005101.4(ISG15):c.379G>T (p.Glu127Ter), Typ...",P,GERM,SUBST,1,0,161454,nAn,1,1014359.0,161454.0,G,T,.,.,AF_EXAC=0.00001;ALLELEID=171288;CLNDISDB=MONDO...,chr1,Pathogenic,nAn,nAn,C,A,+
8,chr1,1041581,1041582,C>T,0,.,1041581,1041582,"210,0,0",1,1,0,126556|NM_198576.4(AGRN):c.1057C>T (p.Gln353Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,nonsense,587777299.0,nAn,RCV000114428|RCV000235030,N,"Myasthenic syndrome,congenital,8|Congenital my...","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""g.1041582C>T"", ""-""], [""g.26460C>T"", ""-""], [...",nAn,2,"Jul 14,2016",nAn,"ClinGen:CA151199, <a target=_blank href='http:...","NM_198576.4(AGRN):c.1057C>T (p.Gln353Ter), Typ...",P,GERM,SUBST,1,0,126556,nAn,1,1041582.0,126556.0,C,T,.,.,"ALLELEID=132069;CLNDISDB=MONDO:MONDO:0014052,M...",chr1,Pathogenic,nAn,nAn,G,A,+
9,chr1,1042052,1042053,C>G,1,.,1042052,1042053,"210,0,0",1,1,0,644955|NM_198576.4(AGRN):c.1275C>G (p.Tyr425Ter),Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,nonsense,1239736447.0,nAn,RCV000798959,N,"Myasthenic syndrome,congenital,8","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.1275C>G"", ""-""], [""g.1042053C>G"", ""-""], [""...",nAn,1,"Jan 02,2019",nAn,nAn,"NM_198576.4(AGRN):c.1275C>G (p.Tyr425Ter), Typ...",P,GERM,SUBST,1,1,644955,nAn,1,1042053.0,644955.0,C,G,.,.,"ALLELEID=626510;CLNDISDB=MONDO:MONDO:0014052,M...",chr1,Pathogenic,nAn,nAn,A,A,+
11,chr1,1044110,1044111,G>T,1,.,1044110,1044111,"210,0,0",1,1,0,1076092|NM_198576.4(AGRN):c.2002G>T (p.Glu668Ter),Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,nonsense,nAn,nAn,RCV001389873,N,"Myasthenic syndrome,congenital,8","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""-"", ""-""], [""g.1044111G>T"", ""-""], [""g.28989G...",nAn,1,"May 22,2020",nAn,nAn,"NM_198576.4(AGRN):c.2002G>T (p.Glu668Ter), Typ...",P,GERM,SUBST,1,1,1076092,nAn,1,1044111.0,1076092.0,G,T,.,.,"ALLELEID=1058371;CLNDISDB=MONDO:MONDO:0014052,...",chr1,Pathogenic,nAn,nAn,C,A,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

from /home/alu/arield/python_scripts/ariel_merge.ipynb

In [24]:
# Creating a gene id column that we can use to merge the GTF database
clinvar_df['gene_id'] = clinvar_df['origName'].str.split('|').str[1]
clinvar_df['gene_id'] = clinvar_df['gene_id'].str.split('(').str[0]
clinvar_df

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand,gene_id
0,chr1,943994,943995,C>T,1,.,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,+,NM_001385641.1
1,chr1,964511,964512,C>A,0,.,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,+,NM_198317.3
2,chr1,1013982,1013983,G>A,1,.,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,+,NM_005101.4
3,chr1,1014142,1014143,C>T,0,.,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,+,NM_005101.4
4,chr1,1014227,1014228,G>A,1,.,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser83Asn""], [""c.248G>A"",...",nAn,4,"Sep 22,2021",nAn,ClinGen:CA507658,"NM_005101.4(ISG15):c.248G>A (p.Ser83Asn), Type...",CF,GERM,SUBST,1,1,402986,nAn,1,1014228.0,402986.0,G,A,.,.,AF_ESP=0.40158;AF_EXAC=0.37025;AF_TGP=0.33886;...,chr1,Conflicting_interpretations_of_pathogenicity,"Benign(3),Pathogenic(1)",yes,A,C,+,NM_005101.

In [25]:
# Checking that indeed a gene id has been created for all rows
clinvar_df[clinvar_df['gene_id'].isna()]

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand,gene_id


Leaving the stop codon lines in the GTF file so that we can find the codon for stop lost mutations

In [26]:
# Reading the GTF file
gtf_file = "/private5/Projects/dadush/clinvar_base_editing/refseq_hg38_exon_cds"
col_names = ['chrom','type','chromStart','chromEnd','strand','loc_in_frame','gene_id_and_transcript_id']
gtf_df = pd.read_csv(gtf_file, sep='\t',usecols=[0,2,3,4,6,7,8] ,names=col_names)
gtf_df

,chrom,type,chromStart,chromEnd,strand,loc_in_frame,gene_id_and_transcript_id
0,chr1,stop_codon,67093580,67093582,-,.,"gene_id ""NM_001276352.2""; transcript_id ""NM_00..."
1,chr1,CDS,67093583,67093604,-,1,"gene_id ""NM_001276352.2""; transcript_id ""NM_00..."
2,chr1,exon,67092165,67093604,-,.,"gene_id ""NM_001276352.2""; transcript_id ""NM_00..."
3,chr1,CDS,67096252,67096321,-,2,"gene_id ""NM_001276352.2""; transcript_id ""NM_00..."
4,chr1,exon,67096252,67096321,-,.,"gene_id ""NM_001276352.2""; transcript_id ""NM_00..."
...,...,...,...,...,...,...,...
1847991,chr22_KI270734v1_random,CDS,156289,156497,-,0,"gene_id ""NM_001368249.2""; transcript_id ""NM_00..."
1847992,chr22_KI270734v1_random,exon,156289,156497,-,.,"gene_id ""NM_001368249.2""; transcript_id ""NM_00..."
1847993,chr22_KI270734v1_random,CDS,161314,161586,-,0,"gene_id ""NM_001368249.2""; transcript_id ""NM_00..."
1847994,chr22_KI270734v1_random,start_codon,161584,161586,-,.,"gene_id ""NM_001368249.2""; transcript_id ""NM_00..."


In [29]:
# create two new columns by spliting one exists
gtf_df['gene_id'] = gtf_df['gene_id_and_transcript_id'].str.split(';').str[0]
gtf_df['transcript_id'] = gtf_df['gene_id_and_transcript_id'].str.split(';').str[1]
gtf_df['gene_id'] = gtf_df['gene_id'].str.split('"').str[1]
gtf_df['transcript_id'] = gtf_df['transcript_id'].str.split('"').str[1]
gtf_df = gtf_df.drop('gene_id_and_transcript_id', axis=1)
gtf_df

,chrom,type,chromStart,chromEnd,strand,loc_in_frame,gene_id,transcript_id
0,chr1,stop_codon,67093580,67093582,-,.,NM_001276352.2,NM_001276352.2
1,chr1,CDS,67093583,67093604,-,1,NM_001276352.2,NM_001276352.2
2,chr1,exon,67092165,67093604,-,.,NM_001276352.2,NM_001276352.2
3,chr1,CDS,67096252,67096321,-,2,NM_001276352.2,NM_001276352.2
4,chr1,exon,67096252,67096321,-,.,NM_001276352.2,NM_001276352.2
...,...,...,...,...,...,...,...,...
1847991,chr22_KI270734v1_random,CDS,156289,156497,-,0,NM_001368249.2,NM_001368249.2
1847992,chr22_KI270734v1_random,exon,156289,156497,-,.,NM_001368249.2,NM_001368249.2
1847993,chr22_KI270734v1_random,CDS,161314,161586,-,0,NM_001368249.2,NM_001368249.2
1847994,chr22_KI270734v1_random,start_codon,161584,161586,-,.,NM_001368249.2,NM_001368249.2


In [30]:
# Select from the GTF file only the line that is a stop codon or CDS
gtf_df_cds_and_stop_codon = gtf_df[(gtf_df['type']=='stop_codon')|(gtf_df['type']=='CDS')]
gtf_df_cds_and_stop_codon

,chrom,type,chromStart,chromEnd,strand,loc_in_frame,gene_id,transcript_id
0,chr1,stop_codon,67093580,67093582,-,.,NM_001276352.2,NM_001276352.2
1,chr1,CDS,67093583,67093604,-,1,NM_001276352.2,NM_001276352.2
3,chr1,CDS,67096252,67096321,-,2,NM_001276352.2,NM_001276352.2
5,chr1,CDS,67103238,67103382,-,0,NM_001276352.2,NM_001276352.2
7,chr1,CDS,67111577,67111644,-,2,NM_001276352.2,NM_001276352.2
...,...,...,...,...,...,...,...,...
1847985,chr22_KI270734v1_random,CDS,148414,148478,-,2,NM_001368249.2,NM_001368249.2
1847987,chr22_KI270734v1_random,CDS,150350,150499,-,2,NM_001368249.2,NM_001368249.2
1847989,chr22_KI270734v1_random,CDS,150987,151021,-,1,NM_001368249.2,NM_001368249.2
1847991,chr22_KI270734v1_random,CDS,156289,156497,-,0,NM_001368249.2,NM_001368249.2


In [32]:
# merge the two databases by the column of the chromosome and gene id and by providing that only the mutation falls within this CDS
merged_3 = clinvar_df.merge(
    clinvar_df.merge(
        gtf_df_cds_and_stop_codon, on=["gene_id", "chrom"], suffixes=("_clinvar", "_gtf"), how="left"
    ).query("chromStart_gtf <= chromEnd_clinvar <= chromEnd_gtf"),
    how="left",
)

merged_3

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand,gene_id,chromStart_clinvar,chromEnd_clinvar,strand_clinvar,type_clinvar,type_gtf,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id
0,chr1,943994,943995,C>T,1,.,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,+,NM_001385641.1,943994.0,943995.0,.,single nucleotide variant,CDS,943908.0,944150.0,+,0,NM_001385641.1
1,chr1,964511,964512,C>A,0,.,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,+,NM_198317.3,964511.0,964512.0,.,single nucleotide variant,CDS,964349.0,964530.0,+,0,NM_198317.3
2,chr1,1013982,1013983,G>A,1,.,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,+,NM_005101.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,1014142,1014143,C>T,0,.,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,+,NM_005101.4,1014142.0,1014143.0,.,single nucleotide variant,CDS,1013984.0,1014475.0,+,0,NM_005101.4
4,chr1,1014227,1014228,G>A,1,.,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen

In [38]:
# There are 8 rows for which merging did not work
merged_3[(merged_3['chromStart_gtf'].isna())&(merged_3['molConseq']=='stop lost')]

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand,gene_id,chromStart_clinvar,chromEnd_clinvar,strand_clinvar,type_clinvar,type_gtf,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id
31579,chr16,173597,173598,T>G,2,.,173597,173598,"210,0,0",1,1,0,15653|NM_000517.4(HBA2):c.427T>G (p.Ter143Glu),Pathogenic/Likely pathogenic,&#9733;&#9733;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,3040|HBA2,stop lost,41464951.0,nAn,RCV000016939|RCV000759054|RCV001285922|RCV0012...,N,HEMOGLOBIN SEAL ROCK|not provided|none provide...,|MedGen:CN517202|MedGen:CN235283|MONDO:MONDO:0...,germline,GRCh38,16p13.3,"[[""g.1948T>G"", ""-""], [""-"", ""-""], [""-"", ""-""], [...",nAn,5,"Mar 23,2020",nAn,"HBVAR:706, ClinGen:CA125597, <a target=_blank ...","NM_000517.4(HBA2):c.427T>G (p.Ter143Glu), Type...",LP,GERM,SUBST,1,2,15653,nAn,16,173598.0,15653.0,T,G,.,.,"ALLELEID=30692;CLNDISDB=.|MONDO:MONDO:0011399,...",chr16,Pathogenic/Likely_pathogenic,nAn,nAn,T,A,+,NM_000517.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31580,chr16,173597,173598,T>A,1,.,173597,173598,"210,0,0",1,1,0,15626|NM_000517.4(HBA2):c.427T>A (p.Ter143Lys),Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,3040|HBA2,stop lost,41464951.0,nAn,RCV000016893|RCV000022603|RCV000985726,N,"HEMOGLOBIN ICARIA|Hemoglobin H disease,nondele...",|MedGen:C3279561|MedGen:CN517202,germline,GRCh38,16p13.3,"[[""g.1948T>A"", ""-""], [""-"", ""-""], [""g.173598T>A...",nAn,2,"Jun 25,2019",nAn,"HBVAR:704, ClinGen:CA125550, <a target=_blank ...","NM_000517.4(HBA2):c.427T>A (p.Ter143Lys), Type...",P,GERM,SUBST,1,1,15626,nAn,16,173598.0,15626.0,T,A,.,.,ALLELEID=30665;CLNDISDB=.|MedGen:C3279561|MedG...,chr16,Pathogenic,nAn,nAn,T,A,+,NM_000517.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31581,chr16,173597,173598,T>C,2,.,173597,173598,"210,0,0",1,1,0,15624|NM_000517.4(HBA2):c.427T>C (p.Ter143Gln),Pathogenic,&#9733;&#9733;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,3040|HBA2,stop lost,41464951.0,nAn,RCV000016891|RCV000022602|RCV000169546|RCV0005...,Y,Hemoglobin constant spring|Hemoglobin H diseas...,MedGen:C3891114|MedGen:C3279561|MONDO:MONDO:00...,"germline,inherited,unknown",GRCh38,16p13.3,"[[""g.1948T>C"", ""-""], [""-"", ""-""], [""c.427T>C"", ...",nAn,9,"Oct 15,2020",nAn,"HBVAR:703, ClinGen:CA125546, <a target=_blank ...","NM_000517.4(HBA2):c.427T>C (p.Ter143Gln), Type...",P,GERM,SUBST,1,2,15624,nAn,16,173598.0,15624.0,T,C,.,.,AF_EXAC=0.00005;AF_TGP=0.00020;ALLELEID=30663;...,chr16,Pathogenic,nAn,nAn,T,A,+,NM_000517.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31583,chr16,173599,173600,A>T,2,.,173599,173600,"210,0,0",1,1,0,15652|NM_000517.4(HBA2):c.429A>T (p.Ter143Tyr),Pathogenic,&#9733;&#9733;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,3040|HBA2,stop lost,41412046.0,nAn,RCV000016938|RCV000508511|RCV001282879,N,"Hemoglobin H disease,nondeletional|not provide...",MedGen:C3279561|MedGen:CN517202|MedGen:CN235283,germline,GRCh38,16p13.3,"[[""-"", ""-""], [""g.1950A>T"", ""-""], [""g.173600A>T...",nAn,3,"Aug 24,2020",nAn,"HBVAR:707, ClinGen:CA125595, <a target=_blank ...","NM_000517.4(HBA2):c.429A>T (p.Ter143Tyr), Type...",P,GERM,SUBST,1,2,15652,nAn,16,173600.0,15652.0,A,T,.,.,ALLELEID=30691;CLNDISDB=MedGen:C3279561|MedGen...,chr16,Pathogenic,nAn,nAn,A,G,+,NM_000517.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50174,chr2,26458041,26458042,T>C,0,.,26458041,26458042,"210,0,0",1,1,0,916017|NM_194248.3(OTOF):c.*196T>C,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&

In [45]:
# The selection of the rpws of the mutation that are stop lost and the position of the coordinate falls only within the stop codon of GTF
stop_lost_df = merged_3[(merged_3['molConseq']=='stop lost')&(merged_3['type_gtf']=='stop_codon')]
stop_lost_df

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand,gene_id,chromStart_clinvar,chromEnd_clinvar,strand_clinvar,type_clinvar,type_gtf,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id
324,chr1,9982698,9982699,T>C,0,.,9982698,9982699,"210,0,0",1,1,0,37132|NM_022787.4(NMNAT1):c.838T>C (p.Ter280Gln),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,64802|NMNAT1,stop lost,387907290.0,nAn,RCV000030763,N,Leber congenital amaurosis 9,"MONDO:MONDO:0012056, <a target=_blank href='ht...",germline,GRCh38,1p36.22,"[[""g.9982699T>C"", ""-""], [""g.44272T>C"", ""-""], [...",nAn,1,"Sep 01,2012",nAn,<a target=_blank href='http://www.omim.org/ent...,"NM_022787.4(NMNAT1):c.838T>C (p.Ter280Gln), Ty...",P,GERM,SUBST,1,0,37132,nAn,1,9982699.0,37132.0,T,C,.,.,"ALLELEID=45793;CLNDISDB=MONDO:MONDO:0012056,Me...",chr1,Pathogenic,nAn,nAn,A,A,+,NM_022787.4,9982698.0,9982699.0,.,single nucleotide variant,stop_codon,9982699.0,9982701.0,+,.,NM_022787.4
411,chr1,11790681,11790682,T>C,0,.,11790681,11790682,"210,0,0",1,1,0,187906|NM_005957.5(MTHFR):c.1969T>C (p.Ter657Arg),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,4524|MTHFR,stop lost,768434408.0,nAn,RCV000167626,N,Homocystinuria due to methylene tetrahydrofola...,"MONDO:MONDO:0009353, <a target=_blank href='ht...",germline,GRCh38,1p36.22,"[[""-"", ""-""], [""-"", ""p.Ter657Arg""], [""g.1179068...",nAn,1,nAn,nAn,ClinGen:CA198664,"NM_005957.5(MTHFR):c.1969T>C (p.Ter657Arg), Ty...",P,GERM,SUBST,1,0,187906,nAn,1,11790682.0,187906.0,A,G,.,.,AF_EXAC=0.00001;ALLELEID=185757;CLNDISDB=MONDO...,chr1,Pathogenic,nAn,nAn,A,G,-,NM_005957.5,11790681.0,11790682.0,.,single nucleotide variant,stop_codon,11790680.0,11790682.0,-,.,NM_005957.5
2208,chr1,47417272,47417273,T>C,0,.,47417272,47417273,"210,0,0",1,1,0,1077110|NM_012186.3(FOXE3):c.958T>C (p.Ter320Arg),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,2301|FOXE3,stop lost,nAn,nAn,RCV001391241,N,"Cataract 34,multiple types","MONDO:MONDO:0013067, <a target=_blank href='ht...",germline,GRCh38,1p33,"[[""g.6202T>C"", ""-""], [""c.958T>C"", ""p.Ter320Arg...",nAn,1,"May 01,2021",nAn,nAn,"NM_012186.3(FOXE3):c.958T>C (p.Ter320Arg), Typ...",P,GERM,SUBST,1,0,1077110,nAn,1,47417273.0,1077110.0,T,C,.,.,"ALLELEID=1065749;CLNDISDB=MONDO:MONDO:0013067,...",chr1,Pathogenic,nAn,nAn,G,G,+,NM_012186.3,47417272.0,47417273.0,.,single nucleotide variant,stop_codon,47417273.0,47417275.0,+,.,NM_012186.3
2209,chr1,47417273,47417274,G>C,0,.,47417273,47417274,"210,0,0",1,1,0,1077109|NM_012186.3(FOXE3):c.959G>C (p.Ter320Ser),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,2301|FOXE3,stop lost,nAn,nAn,RCV001391240,N,"Cataract 34,multiple types","MONDO:MONDO:0013067, <a target=_blank href='ht...",germline,GRCh38,1p33,"[[""g.6203G>C"", ""-""], [""c.959G>C"", ""p.Ter320Ser...",nAn,1,"May 01,2021",nAn,nAn,"NM_012186.3(FOXE3):c.959G>C (p.Ter320Ser), Typ...",P,GERM,SUBST,1,0,1077109,nAn,1,47417274.0,1077109.0,G,C,.,.,"ALLELEID=1065750;CLNDISDB=MONDO:MONDO:0013067,...",chr1,Pathogenic,nAn,nAn,T,A,+,NM_012186.3,47417273.0,47417274.0,.,single nucleotide variant,stop_codon,47417273.0,47417275.0,+,.,NM_012186.3
2210,chr1,47417273,47417274,G>T,0,.,47417273,47417274,"210,0,0",1,1,0,30157|NM_012186.3(FOXE3):c.959G>T (p.Ter320Leu),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,2301|FOXE3,stop lost,387906793.0,nAn,RCV000023071,N,Congenit

In [46]:
# The selection caused those rows that do not have a mergeing to GTF to drop
stop_lost_df[stop_lost_df['transcript_id'].isna()]

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand,gene_id,chromStart_clinvar,chromEnd_clinvar,strand_clinvar,type_clinvar,type_gtf,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id


In [47]:
# Selection of columns to create a BED format
bed_stop_lost_df = stop_lost_df[['chrom','chromStart_gtf','chromEnd_gtf','transcript_id','ID','strand_gtf']].copy()
bed_stop_lost_df = bed_stop_lost_df.fillna(-1)
#Convert to an integer
bed_stop_lost_df['chromStart_gtf']=bed_stop_lost_df['chromStart_gtf'].astype(int)
bed_stop_lost_df['chromEnd_gtf']=bed_stop_lost_df['chromEnd_gtf'].astype(int)
bed_stop_lost_df['ID']=bed_stop_lost_df['ID'].astype(int)
bed_stop_lost_df

,chrom,chromStart_gtf,chromEnd_gtf,transcript_id,ID,strand_gtf
324,chr1,9982699,9982701,NM_022787.4,37132,+
411,chr1,11790680,11790682,NM_005957.5,187906,-
2208,chr1,47417273,47417275,NM_012186.3,1077110,+
2209,chr1,47417273,47417275,NM_012186.3,1077109,+
2210,chr1,47417273,47417275,NM_012186.3,30157,+
...,...,...,...,...,...,...
97988,chrX,154030367,154030369,NM_001110792.2,143490,-
97989,chrX,154030367,154030369,NM_001110792.2,834679,-
98449,chrX,154564459,154564461,NM_001099857.5,418983,+
98450,chrX,154564459,154564461,NM_001099857.5,11450,+


In [48]:
# save stop lost bed
bed_stop_lost_df.to_csv("/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/stop_lost_only_143_rows.bed", index=False,sep="\t")

mergeing to GTF (except for stop lost mutations which we dealt with above)

In [90]:
# Reading the GTF file
gtf_file = "/private5/Projects/dadush/clinvar_base_editing/gtf.tsv"
gtf_df = pd.read_csv(gtf_file, sep='\t')
gtf_df

,chrom,chromStart,chromEnd,strand,loc_in_frame,gene_id,transcript_id
0,chr1,67093583,67093604,-,1,NM_001276352.2,NM_001276352.2
1,chr1,67096252,67096321,-,2,NM_001276352.2,NM_001276352.2
2,chr1,67103238,67103382,-,0,NM_001276352.2,NM_001276352.2
3,chr1,67111577,67111644,-,2,NM_001276352.2,NM_001276352.2
4,chr1,67115352,67115464,-,1,NM_001276352.2,NM_001276352.2
...,...,...,...,...,...,...,...
749372,chr22_KI270734v1_random,148414,148478,-,2,NM_001368249.2,NM_001368249.2
749373,chr22_KI270734v1_random,150350,150499,-,2,NM_001368249.2,NM_001368249.2
749374,chr22_KI270734v1_random,150987,151021,-,1,NM_001368249.2,NM_001368249.2
749375,chr22_KI270734v1_random,156289,156497,-,0,NM_001368249.2,NM_001368249.2


In [91]:
# merge the two databases by the column of the chromosome and gene id and by providing that only the mutation falls within this CDS
merged_2 = clinvar_df.merge(
    clinvar_df.merge(
        gtf_df, on=["gene_id", "chrom"], suffixes=("_clinvar", "_gtf"), how="left"
    ).query("chromStart_gtf <= chromEnd_clinvar <= chromEnd_gtf"),
    how="left",
)

merged_2

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand,gene_id,chromStart_clinvar,chromEnd_clinvar,strand_clinvar,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id
0,chr1,943994,943995,C>T,1,.,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,+,NM_001385641.1,943994.0,943995.0,.,943908.0,944150.0,+,0.0,NM_001385641.1
1,chr1,964511,964512,C>A,0,.,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,+,NM_198317.3,964511.0,964512.0,.,964349.0,964530.0,+,0.0,NM_198317.3
2,chr1,1013982,1013983,G>A,1,.,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,+,NM_005101.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,1014142,1014143,C>T,0,.,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,+,NM_005101.4,1014142.0,1014143.0,.,1013984.0,1014475.0,+,0.0,NM_005101.4
4,chr1,1014227,1014228,G>A,1,.,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser83

In [92]:
# Calculation of how many nucleotides there are before and after the mutation in each CDS according to the strand on which the gene sits. In addition the calculation of the position within the codon
merged_2["CDS_bases_before_mutation"] = np.where(merged_2["strand_clinvar"]=='+',merged_2["chromEnd_clinvar"]-merged_2["chromStart_gtf"],merged_2["chromEnd_gtf"]-merged_2["chromEnd_clinvar"])
merged_2["CDS_bases_after_mutation"] = np.where(merged_2["strand_clinvar"]=='+',merged_2["chromEnd_gtf"]-merged_2["chromEnd_clinvar"],merged_2["chromEnd_clinvar"]-merged_2["chromStart_gtf"])
merged_2["loc_in_three_nuc"] = (merged_2["CDS_bases_before_mutation"]-merged_2["loc_in_frame"])%3
#merged_2
#minus_df = merged_2[merged_2["strand"]=="-"]
#minus_df.head(n=50)
merged_2 = merged_2.drop('chromStart_clinvar',1)
merged_2 = merged_2.drop('chromEnd_clinvar',1)
merged_2 = merged_2.drop('strand_clinvar',1)
merged_2

/tmp/ipykernel_38047/1002542630.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged_2 = merged_2.drop('chromStart_clinvar',1)
/tmp/ipykernel_38047/1002542630.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged_2 = merged_2.drop('chromEnd_clinvar',1)
/tmp/ipykernel_38047/1002542630.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged_2 = merged_2.drop('strand_clinvar',1)


,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand,gene_id,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc
0,chr1,943994,943995,C>T,1,.,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,+,NM_001385641.1,943908.0,944150.0,+,0.0,NM_001385641.1,155.0,87.0,2.0
1,chr1,964511,964512,C>A,0,.,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,+,NM_198317.3,964349.0,964530.0,+,0.0,NM_198317.3,18.0,163.0,0.0
2,chr1,1013982,1013983,G>A,1,.,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,+,NM_005101.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,1014142,1014143,C>T,0,.,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,+,NM_005101.4,1013984.0,1014475.0,+,0.0,NM_005101.4,332.0,159.0,2.0
4,chr1,1014227,1014228,G>A,1,.,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser8

In [93]:
# merged_2['loc_in_frame'] = merged_2['loc_in_frame'].astype('Int64')
# merged_2['CDS_bases_before_mutation'] = merged_2['CDS_bases_before_mutation'].astype('Int64')
# merged_2['CDS_bases_after_mutation'] = merged_2['CDS_bases_after_mutation'].astype('Int64')
# merged_2['loc_in_three_nuc'] = merged_2['loc_in_three_nuc'].astype('Int64')
# merged_2

In [94]:
# [merged_2['transcript_id'].isna()]merged_2

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,Strand,gene_id,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc
2,chr1,1013982,1013983,G>A,1,.,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,+,NM_005101.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,chr1,1043196,1043197,G>C,0,.,1043196,1043197,"210,0,0",1,1,0,1284256|NM_198576.4(AGRN):c.1385-42G>C,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,intron variant,nAn,nAn,RCV001698933,N,"Myasthenic syndrome,congenital,8","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""g.1043197G>C"", ""-""], [""g.28075G>C"", ""-""], [...",nAn,1,"Sep 22,2021",nAn,nAn,"NM_198576.4(AGRN):c.1385-42G>C, Type: single n...",P,GERM,SUBST,1,0,1284256,nAn,1,1043197.0,1284256.0,G,C,.,.,"ALLELEID=1274111;CLNDISDB=MONDO:MONDO:0014052,...",chr1,Pathogenic,nAn,nAn,A,G,+,NM_198576.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,chr1,1336515,1336516,G>A,0,.,1336515,1336516,"210,0,0",1,1,0,981468|NM_001330311.2(DVL1):c.1715-1G>A,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,1855|DVL1,splice acceptor variant,nAn,nAn,RCV001353069,N,"Robinow syndrome,autosomal dominant 2","MONDO:MONDO:0014591, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""c.1640-1G>A"", ""-""], [""g.1336516C>T"", ""-""], ...",nAn,1,"Oct 22,2019",nAn,nAn,"NM_001330311.2(DVL1):c.1715-1G>A, Type: single...",LP,UNK,SUBST,1,0,981468,nAn,1,1336516.0,981468.0,C,T,.,.,"ALLELEID=969551;CLNDISDB=MONDO:MONDO:0014591,M...",chr1,Likely_pathogenic,nAn,nAn,A,G,-,NM_001330311.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,chr1,1517412,1517413,G>A,1,.,1517412,1517413,"210,0,0",1,1,0,986833|NM_001170535.3(ATAD3A):c.384+1G>A,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,55210|ATAD3A,splice donor variant,nAn,nAn,RCV001267961,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.1517413G>A"", ""-""], [""g.10271G>A"", ""-""], [...",nAn,1,"Oct 23,2020",nAn,nAn,"NM_001170535.3(ATAD3A):c.384+1G>A, Type: singl...",LP,GERM,SUBST,1,1,986833,nAn,1,1517413.0,986833.0,G,A,.,.,ALLELEID=974951;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Likely_pathogenic,nAn,nAn,C,T,+,NM_001170535.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,chr1,1523837,1523838,G>T,1,.,1523837,1523838,"210,0,0",1,1,0,989289|NM_001170535.3(ATAD3A):c.964-1G>T,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,55210|ATAD3A,splice acceptor variant,nAn,nAn,RCV001270778,N,Harel-Yoon syndrome,"MONDO:MONDO:0014958, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""c.1108-1G>T"", ""-""], [""g.1523838G>T"", ""-""], ...",nAn,1,"May 21,2019",nAn,nAn,"NM_001170535.3(ATAD3A):c.964-1G>T, Type: singl...",LP,UNK,SUBST,1,1,989289,nAn,1,1523838.0,989289.0,G,T,.,.,"ALLELEID=977169;CLNDISDB

In [95]:
# Because there is editing in the DNA, i don't drop non-protein coding regions
# merged_2_without_na = merged_2[~(merged_2['transcript_id'].isna())]
# merged_2_without_na

In [96]:
# Exchange between a column containing the strand, so that we get a BED format
cols = list(merged_2)
cols[5], cols[55] = cols[55], cols[5]
merged_2_swap = merged_2.reindex(columns=cols)
merged_2_swap


,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,943908.0,944150.0,+,0.0,NM_001385641.1,155.0,87.0,2.0
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,964349.0,964530.0,+,0.0,NM_198317.3,18.0,163.0,0.0
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,1013984.0,1014475.0,+,0.0,NM_005101.4,332.0,159.0,2.0
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser8

In [97]:
# Filling with actual garbage value all the rows where there was no connection with GTF
merged_2_swap = merged_2_swap.fillna('Nan')
merged_2_swap

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,chromStart_gtf,chromEnd_gtf,strand_gtf,loc_in_frame,transcript_id,CDS_bases_before_mutation,CDS_bases_after_mutation,loc_in_three_nuc
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,943908.0,944150.0,+,0.0,NM_001385641.1,155.0,87.0,2.0
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,964349.0,964530.0,+,0.0,NM_198317.3,18.0,163.0,0.0
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,1013984.0,1014475.0,+,0.0,NM_005101.4,332.0,159.0,2.0
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser8

In [99]:
# save the file
merged_2_swap.to_csv("/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_without_merged_ver_1_with_DNA_seq.tsv", index=False,sep="\t")